# DSP Lab 2: Clocking, Frequency and Power

### EE 16B: Designing Information Devices and Systems II, Fall 2015

**Name 1**:

**Login**: ee16b-


**Name 2**:

**Login**: ee16b-


##Table of Contents

* [Introduction](#intro)
* [Task 1: Ring Oscillator](#task1)
* [Task 2: Clock Frequency](#task2)
* [Task 3: Power Consumption](#task3)

<a id='intro'></a>
## Lab Overview
Last week we built both a DAC and an ADC to interface real world signals with your computer. Now that we have some information in a format we can work with, we can build a digital system that processes our information. This week we will start by building a clock, which gives our digital circuit some notion of time. Without a clock, we can only work with combinational logic; with a clock we can build circuits like registers which enables sequential logic. After we have a clock, we can build a digital signal processing (DSP) module - which is implemented in the MSP430 Launchpad for our purposes - and see how changing certain parameters of the circuit affect other aspects of the circuit.

<a id='task1'></a>
##<span style="color:blue">Task 1: Ring Oscillator</span>

###Materials
- MSP430 Launchpad + USB cable
- <a href="http://www.ti.com/lit/ds/symlink/sn74ls04.pdf">SN7404</a> Inverter
- 5 1$\mu$F Capacitors
- 5 100nF Capacitors
- Power supply
- Oscilloscope & Probe

One of the simplest ways we can build a clock is to use a circuit called the ring oscillator. In essence, a ring oscillator connects an odd number of inverters in a loop, so each time a signal goes through a loop it inverts the state of the node.

<img style="width:500px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/ring-osc.png">

**<span style="color:red">Given a delay through an inverter and the capacitance it is driving is $t_d$, what is the frequency of the generated clock?</span>** Hint: start at a node and think about what happens when that node experiences a step from 0V to 5V (assuming all inverters are supplied with 5V).

YOUR ANSWER HERE

We will now try to build a right oscillator using discrete capacitors and an <a href="http://www.ti.com/lit/ds/symlink/sn74ls04.pdf">SN7404</a> inverter IC. The IC contains 6 inverters; in the pin diagram of the IC below, A is the input and Y is the output. In other words, $NA = \overline{NY}$ for $N=1,2,...,6$. Build the 5-inverter ring oscillator above using 5V as $V_{CC}$ and 100nF capacitors (labelled "104" on the component - this means $10 \times 10^4$ pF). <b>Don't forget to limit the current of the power supply to 100mA.</b>

<img style="width:200px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/inv-pins.png">
<center>SN7404 Inverter pin diagram</center>

Now try to show your clock in the oscilloscope by probing any of the nodes. You will see that the shape of the signal is not exactly a square wave and it does not go all the way up to 5V! What went wrong?

<img style="width:600px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/inv-no-buffer.png">

It turns out that your oscilloscope can be modelled as a capacitor in parallel with a large resistor to ground. This means when you attach your oscilloscope to a node you are actually changing the capacitance at that node, creating an imbalanced ring oscillator. How can we fix this? We need to have some sort of buffer to separate the probe from the ring oscillator. Fortunately we have a spare inverter in the IC... Add one more inverter as an inverting buffer for your output voltage.

<img style="width:600px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/ring-osc-buff.png">

Now probe your ring oscillator again, now at the buffered output. Your clock should look better now. Using the oscilloscope, measure the frequency of your clock. Hint: the "Meas" button is helpful!

<img style="width:600px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/inv-buffer.png">

**<span style="color:red">What is your clock frequency?</span>**

YOUR ANSWER HERE

### Varying Supply Voltage

One way we can change the clock frequency is by changing the supply voltage. Try sweeping the supply voltage from 2.5V to 6V in increments of 0.5V. 

**<span style="color:red">What frequencies do you see? Why?</span>**

<table>
<tr>
<td><b>Supply Voltage (V)</b></td>
<td><b>Frequency (Hz)</b></td>
</tr>
<tr>
<td>2.5</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>3</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>3.5</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>4</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>4.5</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>5</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>5.5</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>6</td>
<td>YOUR ANSWER HERE</td>
</tr>
</table>

YOUR ANSWER HERE

### Varying Capacitance

Changing the capacitor values in the circuit above changes the load of each inverter. This means the delay through each inverter will also change.

**<span style="color:red">If we use capacitors that are 10 times as large, approximately what is the frequency you expect to see? </span>**

YOUR ANSWER HERE

Swap out all of the 100nF capacitors with the 1$\mu$F capacitors (labelled "105" on the component) and measure the frequency of your new clock with a 5V supply.

**<span style="color:red">What is your new clock frequency?</span>**

YOUR ANSWER HERE

<a id='task2'></a>
##<span style="color:blue">Task 2: Clock Frequency</span>

###Materials
- MSP430 Launchpad + USB cable
- Oscilloscope & probe

Now that we can build a clock, we can feed the clock to our microcontroller to compute things. Fortunately, the MSP430F5529 already has a (rather complicated) <a href="http://www.ti.com/lit/ug/slau390d/slau390d.pdf">clocking system</a> so we don't need to provide it with anything. On the board, there are 2 external crystal oscillators that are used as a reference for the microcontroller's clock(s).

<b>`decoder_msp.ino`</b> contains an implementation of the Inverse Discrete Cosine Transform (DCT). Just like a Discrete Fourier Transform, the DCT takes time domain data and converts it to frequency domain data, but all the coefficients are real-valued (unlike the complex coefficients of the DFT). The Launchpad code implements the synthesis equation:
$$x[n]= \sum^N_{k=1}w[k]X[k]\cos\left(\frac{\pi(2n-1)(k-1)}{2N}\right)$$
where $x[n]$ is the time sample, $X[k]$ is the $k$-th DCT coefficient and $w[k]$ are some weights defined as $\frac{1}{\sqrt{N}}$ for $k=1$ and $\sqrt{\frac{2}{N}}$ otherwise. In this lab we use $k=16$, so we decompose our audio to 16 different frequencies.

Don't worry too much about the algorithm - everything is implemented for you in the sketch.

We have some DCT coefficients for the wav file `original.wav` in `dct_drmapan.txt`. We want to stream this data a chunk (16*32 samples) at a time into the Launchpad and do the synthesis on the Launchpad, then stream the time domain data back to the PC and play it. Unfortunately, there are some limitations to the hardware - it can only run up to 25MHz - and you don't want to wait for hours to synthesize the audio. You decide to set a 'timeout' functionality in the MSP such that the synthesis happens from lower frequencies to higher frequencies and when a timer stops the computation is halted.

**<span style="color:red">Suppose the synthesis for a chunk of data takes 3 seconds, if you set a timeout for 500ms how many frequencies are successfully synthesized?</span>**

YOUR ANSWER HERE

The function below simulates the MSP's computation given the number of frequencies it successfully synthesizes. Find the lowest number of frequencies you can use without significantly degrading the audio quality.

In [ ]:
import numpy as np
from decoder_host import *

In [ ]:
def simulate_msp(f_end):
    """ Simulates the inverse DCT computation in the MSP. Saves the result to sim_result.wav.
    Parameters:
        f_end: number of DCT coefficients to use; maximum: 16
    Returns: None
    """
    assert f_end <= 16, "Argument to simulate_msp has to be less than 17."
    
    text_file = open("dct_drmapan.txt", "r")
    dct_coeff = np.array(text_file.read().split('\n')[:-1]).astype(float)
    text_file = open("cos_table.txt", "r")
    cos_table = np.array(text_file.read().split('\n')[:-1]).astype(float)
    
    result = np.zeros((len(dct_coeff),1))
    for f in range(f_end):
        for n in range(len(dct_coeff)//16*16):
            t = n % 16
            result[n] = result[n] + dct_coeff[n-t+f]*cos_table[t*16+f]

    save_wav(result, 'sim_result.wav')

In [ ]:
# Simulation output is saved to sim_result.wav. Play it with VLC Player.
simulate_msp(16)

**<span style="color:red">What is the lowest number of frequencies you can use without significantly degrading the audio quality?</span>**

YOUR ANSWER HERE

Now try it for real in hardware! Open <b>`decoder_msp.ino`</b> in Energia and upload the sketch. <b>Use an oscilloscope to probe pin `2.2`, which shows the clock of the microcontroller.</b>

The function `run()` below launches a terminal where you can send commands to the Launchpad. For starters, run:
- "`set_length 1`"
- "`set_timer 100`"

These commands set the function and Launchpad to synthesize 1 chunk of data (16*32 samples) with a timeout value of 100ms. The default clock speed is 25MHz. The Launchpad will tell you how many different frequencies it could synthesize in the time given. Find the timer value that is enough to synthesize the number of frequencies you chose above.

When you have set the timer value appropriately, run the command "`set_length 64`", which will stream the whole 64 chunks of data. The result will be saved as `result.wav`. Play it and make sure it sounds good!

Lastly, given the same timer length, vary your frequency with the command "`set_clock N`" for an `N`-kHz clock. Try using clocks from 25MHz down to 1MHz and note the number of frequencies successfully synthesized. You can run this with less number of chunks if the whole audio file takes too much time.

When you're done, run the command "`quit`" to exit the terminal.

In [ ]:
from decoder_host import *
run()

<a id='task3'></a>
##<span style="color:blue">Task 3: Power Consumption</span>

###Materials
- MSP430 Launchpad + USB cable
- Oscilloscope & probe
- Digital Multimeter

So how much power does it actually take to run the synthesis algorithm on the Launchpad? To measure this, we can use a Digital Multimeter (DMM) as an Ammeter. Your Launchpad has some isolation jumpers below:

<img style="width:300px" src="http://inst.eecs.berkeley.edu/~ee16b/fa15/lab_pics/msp-jumpers.png">

Disconnect the one labelled `3V3` - this jumper supplies power to your microcontroller (be careful not to loose the jumper!). Set the DMM to measure DC current (make sure to use the 2 terminals marked LO and I) and connect the DMM to the `3V3` jumper pins.

Upload the sketch <b>`decoder_msp_powermeas.ino`</b>. This sketch is slightly modified from the one before such that the timer value is 100 times what you request and the computation is repeated over and over until the timer stops. We do this so you can measure the power when the MSP430 is actually doing the computation. Run the `run()` function again, but now do the following commands:
- "`set_time 50`"
- "`set_length 1`"

This will run 5 seconds of decoding with just 1 chunk of data. Record the current and compute the power consumed by the Launchpad, assuming it takes 3.3V supply. Then vary the clock frequency using the command "`set_clock`" and fill in the table below.

<table>
<tr>
<td><b>Frequency (kHz)</b></td>
<td><b>Power (MW)</b></td>
</tr>
<tr>
<td>25000</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>22500</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>20000</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>17500</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>15000</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>12500</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>10000</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>7500</td>
<td>YOUR ANSWER HERE</td>
</tr>
<tr>
<td>5000</td>
<td>YOUR ANSWER HERE</td>
</tr>
</table>

###<b>Remember to put the 3V3 jumper back!</b>

In [ ]:
from decoder_host import *
run()

It turns out that the 3.3V source voltage that you probed is not exactly the voltage used in the microcontroller. The chip contains a voltage regulator that supplies $V_{\text{CORE}}$ to the microcontroller circuit where $V_{\text{CORE}} < 3 $. In order to run the microcontroller at certain frequencies, the voltage regulator needs to supply at least certain voltages. 

The Launchpad sketch contains the following lines in the function `setClock`:

```C
if (freq < 8000) PMM_setVCore(PMM_CORE_LEVEL_0);
else if (freq < 12000) PMM_setVCore(PMM_CORE_LEVEL_1);
else if (freq < 20000) PMM_setVCore(PMM_CORE_LEVEL_2);
else PMM_setVCore(PMM_CORE_LEVEL_3);
```

These lines set the core voltage levels based on the desired frequency. In real digital circuits or processors, this voltage regulation can be done dynamically on-the-run by either the firmware or the hardware itself. For example, when you want to overclock your processor, you would supply a larger voltage to the processor. This concept is known as Dynamic Voltage Scaling.